In [4]:
import pandas as pd
import numpy as np
import sqlite3

In [14]:
con = sqlite3.connect('first_project.db')
data_import = pd.read_sql_query('SELECT * from airplane_accidents', con)

data_import

,id,date,time,location,operator,flight_no,route,ac_type,registration,cn_ln,aboard,fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2; passengers:1; crew:1,1; passengers:1; crew:0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1; passengers:0; crew:1,1; passengers:0; crew:0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5; passengers:0; crew:5,5; passengers:0; crew:5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1; passengers:0; crew:1,1; passengers:0; crew:1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20; passengers:?; crew:?,14; passengers:?; crew:?,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4967,4967,"May 21, 2021",1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11; passengers:7; crew:4,11; passengers:7; crew:4,0,"While on final approach, in poor weather condi..."
4968,4968,"June 10, 2021",800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14; passengers:12; crew:2,12; passengers:11; crew:1,0,The plane was carrying military personnel and ...
4969,4969,"July 04, 2021",11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96; passengers:88; crew:8,50; passengers:?; crew:?,3,"While attempting to land at Jolo Airport, the ..."
4970,4970,"July 06, 2021",1500,"Palana, Russia",\tKamchatka Aviation Enterprise,251,Petropavlovsk - Palana,Antonov An 26B-100,\tRA-26085,12310,28; passengers:22; crew:6,28; passengers:22; crew:6,0,The passenger plane crashed into the top of a ...
